In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = ""

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
    

# Written by Braydon






In [ ]:
db = client['Steam']
games = db['Games Names']
playtime = db['Playtime']
genrestogames = db['Game-Genre Relationships']
genres = db['Genres']
tagtogames = db['Game-Tag Relationships']
cattogames = db['Game-Category Relationships']
tags = db['Tags']
categories = db['Categories']

#written by Braydon

In [ ]:
# data mining stuff

# find games that are often played together
df = pd.DataFrame(list(playtime.find()))
df3 = pd.DataFrame(list(genres.find()))
df4 = pd.DataFrame(list(genrestogames.find()))

df3 = df3.rename(columns = {'_id': 'genreID'})
df4 = pd.merge(df4, df3, on='genreID')


df4 = df4.drop(columns = ['_id'])
df4 = df4.dropna()
df4 = df4.groupby(['gameID', 'genreName']).size().unstack().fillna(0)
df4 = df4.applymap(lambda x: 1 if x > 0 else 0)

#df4 = df4.drop(columns = ['genreID'])

merged_df = pd.merge(df, df4, on='gameID')

merged_df = merged_df.drop(columns = ['playtime'])
merged_df = merged_df.dropna()
merged_df = merged_df.drop(columns = ['_id'])
merged_df = merged_df.drop(columns = ['gameName'])
merged_df = merged_df.drop(columns = ['gameID'])

merged_df = merged_df.groupby(['playerID']).sum()
merged_df = merged_df.applymap(lambda x: 1 if x > 0 else 0)

ddd = association_rules(apriori(merged_df, min_support=0.01, use_colnames=True), metric="confidence", min_threshold=0.8)
ddd.head(5)


#written by Braydon

In [ ]:
ddd.drop(columns = ['antecedent support', 'consequent support', 'zhangs_metric'], inplace = True)

In [ ]:
ddd.head(5)

In [ ]:
fig = px.scatter(
    ddd,
    x="leverage",
    y="confidence",
    size="support",
    color="conviction",
    title="Genres Played Together",
    template="plotly_dark",

)
fig.show()

#written by Braydon

In [ ]:
# find the genres that most commonly go together

df5 = pd.DataFrame(list(games.find()))
df5.rename(columns = {'_id': 'gameID'}, inplace = True)

merged_df = pd.merge(df5, df4, on='gameID')
merged_df = merged_df.drop(columns = ['gameID'])
merged_df = merged_df.drop(columns = ['gameName'])
dd = association_rules(apriori(merged_df, min_support=0.001, use_colnames=True), metric="confidence", min_threshold=0.5)
dd.drop(columns = ['antecedent support', 'consequent support', 'zhangs_metric'], inplace = True)
dd.head(5)

#written by Braydon

In [ ]:
fig = px.scatter(
    dd,
    x="leverage",
    y="confidence",
    size="support",
    color="conviction",
    title="Genres Grouped Together",

)
fig.show()

#written by Braydon

In [ ]:
# data mining stuff

# find games that are often played together
df = pd.DataFrame(list(playtime.find()))
df3 = pd.DataFrame(list(tags.find()))
df4 = pd.DataFrame(list(tagtogames.find()))

df3 = df3.rename(columns = {'_id': 'tagID'})
df4 = pd.merge(df4, df3, on='tagID')


df4 = df4.drop(columns = ['_id'])
df4 = df4.dropna()
df4 = df4.groupby(['gameID', 'tagName']).size().unstack().fillna(0)
df4 = df4.applymap(lambda x: 1 if x > 0 else 0)

#df4 = df4.drop(columns = ['genreID'])

merged_df = pd.merge(df, df4, on='gameID')

merged_df = merged_df.drop(columns = ['playtime'])
merged_df = merged_df.dropna()
merged_df = merged_df.drop(columns = ['_id'])
merged_df = merged_df.drop(columns = ['gameName'])
merged_df = merged_df.drop(columns = ['gameID'])

merged_df = merged_df.groupby(['playerID']).sum()
merged_df = merged_df.applymap(lambda x: 1 if x > 0 else 0)

ddcd = association_rules(apriori(merged_df, min_support=0.1, use_colnames=True), metric="confidence", min_threshold=0.8)
ddcd.drop(columns = ['antecedent support', 'consequent support', 'zhangs_metric'], inplace = True)
ddcd.head(5)

#written by Braydon


In [ ]:
fig = px.scatter(
    ddcd,
    x="leverage",
    y="confidence",
    size="support",
    color="conviction",
    title="Tags Played Together",

)
fig.show()

#written by Braydon

In [ ]:
# find the tags that most commonly go together

df5 = pd.DataFrame(list(games.find()))
df5.rename(columns = {'_id': 'gameID'}, inplace = True)

merged_df = pd.merge(df5, df4, on='gameID')
merged_df = merged_df.drop(columns = ['gameID'])
merged_df = merged_df.drop(columns = ['gameName'])
dd = association_rules(apriori(merged_df, min_support=0.001, use_colnames=True), metric="confidence", min_threshold=0.8)
dd.drop(columns = ['antecedent support', 'consequent support', 'zhangs_metric'], inplace = True)
dd.head(5)

#written by Braydon

In [ ]:
fig = px.scatter(
    dd,
    x="leverage",
    y="confidence",
    size="support",
    color="conviction",
    title="Tags Grouped Together",

)
fig.show()

#written by Braydon

In [ ]:
# categories that players often play together
df = pd.DataFrame(list(playtime.find()))
df3 = pd.DataFrame(list(categories.find()))
df4 = pd.DataFrame(list(cattogames.find()))

df3 = df3.rename(columns = {'_id': 'categoryID'})
df4 = pd.merge(df4, df3, on='categoryID')


df4 = df4.drop(columns = ['_id'])
df4 = df4.dropna()
df4 = df4.groupby(['gameID', 'categoryName']).size().unstack().fillna(0)
df4 = df4.applymap(lambda x: 1 if x > 0 else 0)

#df4 = df4.drop(columns = ['genreID'])

merged_df = pd.merge(df, df4, on='gameID')

merged_df = merged_df.drop(columns = ['playtime'])
merged_df = merged_df.dropna()
merged_df = merged_df.drop(columns = ['_id'])
merged_df = merged_df.drop(columns = ['gameName'])
merged_df = merged_df.drop(columns = ['gameID'])

merged_df = merged_df.groupby(['playerID']).sum()
merged_df = merged_df.applymap(lambda x: True if x > 0 else False)


#dcd = association_rules(apriori(merged_df, min_support=0.1, use_colnames=True), metric="confidence", min_threshold=0.8)
#dcd.drop(columns = ['antecedent support', 'consequent support', 'zhangs_metric'], inplace = True)
#dcd.head(5)

# written by Braydon

In [ ]:
fig = px.scatter(
    ddd,
    x="leverage",
    y="confidence",
    size="support",
    color="conviction",
    title="Genres Played Together",

)
fig.show()

#written by Braydon

In [ ]:
# find the categories that most commonly go together

df5 = pd.DataFrame(list(games.find()))
df5.rename(columns = {'_id': 'gameID'}, inplace = True)

merged_df = pd.merge(df5, df4, on='gameID')
merged_df = merged_df.drop(columns = ['gameID'])
merged_df = merged_df.drop(columns = ['gameName'])
dd = association_rules(apriori(merged_df, min_support=0.01, use_colnames=True), metric="confidence", min_threshold=0.5)
dd.drop(columns = ['antecedent support', 'consequent support', 'zhangs_metric'], inplace = True)
dd.head(5)

#written by Braydon

In [ ]:
fig = px.scatter(
    dd,
    x="leverage",
    y="confidence",
    size="support",
    color="conviction",
    title="Categories Grouped Together",

)
fig.show()

#written by Braydon